# Parameter Space

To run a DYNAMITE model, one must specify a number of parameteters for the gravitational potential. The aim of this notebook is to demonstrate how to specify these parameters are specified, and to highlight featues that we have implemented in order to help you explore parameter space. 

We'll start as before by reading the same configuration file as previously,

In [1]:
import dynamite as dyn

# read the config file
fname = 'NGC6278_config.yaml'
c = dyn.config_reader.Configuration(fname, silent=True)

No previous models have been found:
Making an empty table in AllModels.table


When the configuration object is created, internally, a parameter space object is created. This ``parspace`` object is a list, and every entry of this list is a parameter in our model,  Lets extract this and have a look

In [2]:
# extract the parameter space 
parspace = c.parspace
print('type of parspace is', type(parspace))
print('length of parspace is', len(parspace))
print('the parameter names are:')
for par in parspace:
    print('   -', par.name)

type of parspace is <class 'parameter_space.ParameterSpace'>
length of parspace is 8
the parameter names are:
   - mass
   - a
   - dc
   - f
   - q
   - p
   - u
   - ml


Several properties are specified for each parameter in the configuration file. Let's look at the value

In [3]:
print('Parameter / value :')
for par in c.parspace:
    print(f'   {par.name} = {par.value}')

Parameter / value :
   mass = 6.0
   a = 0.001
   dc = 10.0
   f = 1.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 3.0


These are the starting values from which we would like to run a model.

One complication in specifying these values is that, for some parameters, we would like to take logarithmically spaced steps through parameter space, i.e. the ones which are specificed as
```
parameters -> XXX -> logarithmic : True
```
Logarithmic spacing can be useful for mass parameters. For other parameters (e.g. length scales) linearly spaced steps may be more appropriate. For other types of parameters (e.g. angles) a different spacing altogether may be preferable.

To handle these possibilities, we introduce the concept of ``raw`` parameter values, distinct from the values themselves. All values associated with parameter in the configuration file are given in ``raw`` units. When we step through parameter space, we take linear steps in ``raw`` values. The conversion from raw values to the parameter values is handles by the method
```
Parameter.get_par_value_from_raw_value
```
So to convert the above list from raw values, we can do the following,

In [4]:
print('Parameter / value :')
for par in c.parspace:
    raw_value = par.value
    par_value = par.get_par_value_from_raw_value(raw_value)
    print(f'   {par.name} = {par_value}')

Parameter / value :
   mass = 1000000.0
   a = 0.001
   dc = 10.0
   f = 10.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 3.0


Notice how only those parameters which have been specified with ``logarithmic : True`` have been modified.

Another property that we specifie for each parameter is whether or not it is fixed, including a value and whether or not the parameter is fixed,

In [5]:
for par in parspace:
    if par.fixed is True:
        fix_string = ' is fixed.'
    if par.fixed is False:
        fix_string = ' is NOT fixed.'
    print(f'{par.name}{fix_string}')

mass is fixed.
a is fixed.
dc is fixed.
f is NOT fixed.
q is fixed.
p is fixed.
u is fixed.
ml is NOT fixed.


The only parameters which are not fixed for this example are the dark matter fraction ``f`` and the mass-to-light ratio ``ml``. For these free parameters, additional properties about how search through parameter space are stored in the ``par_generator_settings`` attribute,

In [8]:
for par in parspace:
    if par.fixed is False:
        tmp = par.par_generator_settings
        lo, hi, step = tmp['lo'], tmp['hi'], tmp['step']
        print(f'{par.name} takes step-size {step} and bounds ({lo,hi})')

f takes step-size 0.05 and bounds ((0.0, 3.0))
ml takes step-size 1.0 and bounds ((0.8, 7.5))


How do we search over these free parameters? Running models (especially calcuating the orbit library) is expensive, so we will want to search through parameter space in the most efficient way possible.

In general, an algorithm to search through parameter space will take as input
1. the output of all models which have been run so far (e.g. chi-squared values)
2. setting for the free parameters (e.g. step-size and bounds)
The algorithm will then output a new list of parameters for which we want to run models.

In DYNAMITE, we implement this generic idea in the class
``dyn.parameter_sapce.ParameterGenerator``
In the configuration file, you specify *which* parameter generator you would like to use, at the location
```
parameter_space_settings -> generator_type
```
The current choice is 

In [10]:
c.settings.parameter_space_settings['generator_type']

'LegacyGridSearch'

This parameter generator requires an additional setting which is set at,
```
parameter_space_settings -> generator_settings -> threshold_del_chi2
```
which is accessed in the following way,

In [18]:
threshold_del_chi2 = c.settings.parameter_space_settings['generator_settings']['threshold_del_chi2']
print(f'threshold_del_chi2 = {threshold_del_chi2}')

threshold_del_chi2 = 0.5


The algorithm implemented to generate parameters in ``LegacyGridSearch`` is the following,

```
iteration = 0
if iteration == 0
    all parameters take `value` specified in the config
else:
    1. find the model with the lowest chi-squared
    2. find all models with chi-squared within threshold_del_chi2 of the lowest value
    3. for all models satisfying that criteria:
        - for all free parameters:
            - generate a new parameter set +/-1 step-size from the current value
    4. Remove any models with parameters outside specified bounds
    5. iteration = iteration + 1
stop if no new models are added, or any other stopping criteria are met    
```

For those of you who have used the previous version of the trixial Schwarzschild modelling code (aka ``schwpy``), this is the same algorithm which was implemented there.

The last line of the algorithm mentions stopping criteria. Settings which control the stopping criteria are also speicified in the configuration file, under
```
parameter_space_settings -> stopping_criteria
```
The current settings which are the following,

In [28]:
stopping_crierita = c.settings.parameter_space_settings['stopping_criteria']
for key in stopping_crierita:
    print(f'{key} = {stopping_crierita[key]}')

min_delta_chi2 = 0.5
n_max_mods = 50
n_max_iter = 10


These have the following mwaning,

- if no new model impoves the chi-squared by at least ``min_delta_chi2``, then stop
- if we have already run ``n_max_mods`` models, then stop
- if we have already run ``n_max_iter`` iterations, then stop

:)
